In [1]:
from configs.default import _C as config
from configs.default import update_config

from datasets import flickr8k_parse

from keras import backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras.backend.tensorflow_backend import set_session
from models import batch_generator, decoder

import json
import numpy as np
import os
import pandas as pd
import path_generation
import tensorflow as tf
import text_processing
import time

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '

In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12006001025085623202, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1462032793
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6801201856544217791
 physical_device_desc: "device: 0, name: GeForce 840M, pci bus id: 0000:07:00.0, compute capability: 5.0"]

In [3]:
config_file = "./configs/attn.yaml"
update_config(config, config_file)

# Decoder

### Captions encoding

Before building decoder, it is necessary to encode captions into one-hot vectors which further would be used in embedding layer.

### COCO dataset

In [4]:
if config.DATASET == 'Coco':
    if config.ATTENTION:
        features_file_train = "vgg16_coco_train_attn.npy"
        features_file_val = "vgg16_coco_val_attn.npy"
    else:
        features_file_train = "vgg16_coco_train.npy"
        features_file_val = "vgg16_coco_val.npy"
    
    
    val_filenames_with_captions = coco_parse.get_image_filename_with_caption(config.PATH.ANNOTATIONS_PATH, 
                                                                             config.PATH.IMG_PATH, 
                                                                             train=False)

    val_filenames_with_all_captions = coco_parse.get_image_with_all_captions(val_filenames_with_captions)

    train_filenames_with_captions = coco_parse.get_image_filename_with_caption(config.PATH.ANNOTATIONS_PATH, 
                                                                               config.PATH.IMG_PATH,
                                                                               train=True)
    train_filenames_with_all_captions = coco_parse.get_image_with_all_captions(train_filenames_with_captions)

    ### Extract captions
    train_captions = coco_parse.make_list_of_captions(train_filenames_with_all_captions)
    val_captions = coco_parse.make_list_of_captions(val_filenames_with_all_captions)

### Flickr8k dataset

In [5]:
if config.DATASET == 'Flickr8k':
    if config.ATTENTION:
        features_file_train = "vgg16_flickr8k_train_attn.npy"
        features_file_val = "vgg16_flickr8k_val_attn.npy"
    else:
        features_file_train = "vgg16_flickr8k_train.npy"
        features_file_val = "vgg16_flickr8k_val.npy"

    captions_file = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr8k.token.txt")
    train_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.trainImages.txt")
    dev_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.devImages.txt")
    test_txt_path = os.path.join(config.PATH.ANNOTATIONS_PATH, "Flickr_8k.testImages.txt")    
        
    filenames_with_all_captions = flickr8k_parse.generate_filenames_with_all_captions(captions_file, 
                                                                                      config.PATH.IMG_PATH)
    train_filenames_with_all_captions = flickr8k_parse.generate_set(train_txt_path, 
                                                                    filenames_with_all_captions,
                                                                    config.PATH.IMG_PATH)
    val_filenames_with_all_captions = flickr8k_parse.generate_set(dev_txt_path, 
                                                                  filenames_with_all_captions, 
                                                                  config.PATH.IMG_PATH)
    test_filenames_with_all_captions = flickr8k_parse.generate_set(test_txt_path, 
                                                                   filenames_with_all_captions, 
                                                                   config.PATH.IMG_PATH)

    train_captions = flickr8k_parse.make_list_of_captions(train_filenames_with_all_captions)
    val_captions = flickr8k_parse.make_list_of_captions(val_filenames_with_all_captions)

In [6]:
### Preprocess captions
text_processing.preprocess_captions(val_captions)
text_processing.preprocess_captions(train_captions)

In [7]:
### Add markers of captions' starts and ends
text_processing.add_start_and_end_to_captions(train_captions)
text_processing.add_start_and_end_to_captions(val_captions)

In [8]:
### Create vocabulary from the training captions
train_vocab = text_processing.Vocabulary()
for caption_list in train_captions:
    for caption in caption_list:
        tmp_caption_list = caption.split()
        for word in tmp_caption_list:
            train_vocab.add_word(word)

In [9]:
if not os.path.exists(config.PATH.VOCABULARY_PATH):
    os.mkdir(config.PATH.VOCABULARY_PATH)
train_vocab.save_vocabulary(config.VOCABULARY.WORD_TO_ID, config.VOCABULARY.ID_TO_WORD, config.VOCABULARY.COUNT)

In [10]:
train_captions_tokens = text_processing.tokenise_captions(train_captions, train_vocab)
val_captions_tokens = text_processing.tokenise_captions(val_captions, train_vocab)

In [11]:
train_captions_tokens[0]

[[1, 2, 3, 4, 5, 6, 7, 2, 8, 4, 9, 10, 11, 12],
 [1, 3, 4, 13, 14, 4, 15, 11, 12],
 [1, 16, 17, 18, 19, 20, 21, 10, 22, 23, 12],
 [1, 16, 17, 24, 25, 9, 10, 11, 12],
 [1, 16, 17, 6, 15, 2, 26, 27, 28, 29, 30, 12]]

In [12]:
train_captions[0]

['<sos> a black dog is running after a white dog in the snow <eos>',
 '<sos> black dog chasing brown dog through snow <eos>',
 '<sos> two dogs chase each other across the snowy ground <eos>',
 '<sos> two dogs play together in the snow <eos>',
 '<sos> two dogs running through a low lying body of water <eos>']

### Decoder NN

### GRU

In [13]:
path_gen = path_generation.PathGenerator(config.DECODER.GRU, 
                                         config.DATASET, 
                                         config.DECODER.NUM_RNN_LAYERS, 
                                         config.DECODER.BATCH_SIZE, 
                                         config.DECODER.BATCH_NORM, 
                                         config.DECODER.DROPOUT, 
                                         config.ATTENTION, 
                                         config.DECODER.ATTN_TYPE)

path_checkpoint = path_gen.get_weights_path()
model_path = path_gen.get_model_path()
callbacks_path = path_gen.get_callbacks_path()

In [14]:
features_file_train_path = os.path.join(config.PATH.FEATURES_PATH, features_file_train)
features_file_val_path = os.path.join(config.PATH.FEATURES_PATH, features_file_val)

transfer_values = np.load(features_file_train_path)
val_transfer_values = np.load(features_file_val_path)

In [15]:
if config.ATTENTION:
    print(transfer_values.shape)
    transfer_values = transfer_values.reshape(len(train_filenames_with_all_captions), transfer_values.shape[1] ** 2, -1)
    val_transfer_values = val_transfer_values.reshape(len(val_filenames_with_all_captions), val_transfer_values.shape[1] ** 2, -1)
    print(transfer_values.shape)

(6000, 14, 14, 512)
(6000, 196, 512)


In [16]:
decoder_model = decoder.Decoder(config.DECODER.INITIAL_STATE_SIZE,
                                config.DECODER.EMBEDDING_OUT_SIZE,
                                config.DECODER.NUM_RNN_LAYERS,
                                config.DECODER.GRU,
                                config.DECODER.BATCH_NORM,
                                config.DECODER.DROPOUT,
                                config.ATTENTION,
                                config.DECODER.ATTN_TYPE,
                                transfer_values,
                                train_vocab)
decoder_model = decoder_model.build_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
if config.DECODER.GRU:
    generator = batch_generator.generate_batch(transfer_values, 
                                               train_captions_tokens, 
                                               number_of_words=train_vocab.number_of_words, 
                                               batch_size=config.DECODER.BATCH_SIZE)
    val_generator = batch_generator.generate_batch(val_transfer_values, 
                                                   val_captions_tokens, 
                                                   number_of_words=train_vocab.number_of_words, 
                                                   batch_size=config.DECODER.BATCH_SIZE)
else:
    generator = batch_generator.generate_batch(transfer_values, 
                                               train_captions_tokens, 
                                               number_of_words=train_vocab.number_of_words, 
                                               batch_size=config.DECODER.BATCH_SIZE, 
                                               gru=config.DECODER.GRU)
    
    val_generator = batch_generator.generate_batch(val_transfer_values, 
                                                   val_captions_tokens, 
                                                   number_of_words=train_vocab.number_of_words, 
                                                   batch_size=config.DECODER.BATCH_SIZE, 
                                                   gru=config.DECODER.GRU)              

In [18]:
if config.DECODER.OPTIMIZER:
    optimizer = RMSprop(lr=config.DECODER.LR, decay=config.DECODER.DECAY)

In [19]:
decoder_model.compile(optimizer=optimizer,
                      loss=config.DECODER.LOSS)

In [20]:
model_json = decoder_model.to_json()
try:
    os.mkdir(config.PATH.MODELS_ARCHITECTURE_PATH)
except:
    print('The folder already exists')
with open(model_path, "w") as json_file:
    json.dump(json.loads(model_json), json_file, indent=4)

D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'batch_normalization_5/cond/Merge:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'batch_normalization_6/cond/Merge:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword argument

D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_19/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1_19/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_20/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1_20/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'init

The folder already exists


### Checkpoints

During the training process, it is a good idea to save the weights periodically.

In [21]:
try:
    os.mkdir(configs.WEIGHTS_PATH)
except:
    print('The folder already exists')

checkpoints = ModelCheckpoint(path_checkpoint, 
                              verbose=config.DECODER.VERBOSE, 
                              save_weights_only=True, 
                              save_best_only=config.DECODER.SAVE_BEST)

reduce_lr = ReduceLROnPlateau(monitor=config.DECODER.MONITOR, 
                              factor=config.DECODER.FACTOR,
                              patience=config.DECODER.PATIENCE, 
                              verbose=config.DECODER.VERBOSE, 
                              min_lr=config.DECODER.MIN_LR)

The folder already exists


In [22]:
tf_configuration = tf.ConfigProto()
tf_configuration.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_configuration))
start = time.time()
callbacks = decoder_model.fit_generator(generator=generator,
                                        steps_per_epoch=int(len(train_filenames_with_all_captions) / config.DECODER.BATCH_SIZE),
                                        epochs=config.DECODER.EPOCHS,
                                        callbacks=[checkpoints, reduce_lr],
                                        validation_data=val_generator,
                                        validation_steps=config.DECODER.VAL_STEPS)
time_train = time.time() - start

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/20
187/187 [==============================] - ETA: 2:35:08 - loss: 8.98 - ETA: 1:19:35 - loss: 8.09 - ETA: 54:22 - loss: 7.2711 - ETA: 41:44 - loss: 6.67 - ETA: 34:09 - loss: 6.21 - ETA: 29:05 - loss: 5.92 - ETA: 25:27 - loss: 5.62 - ETA: 22:44 - loss: 5.35 - ETA: 20:36 - loss: 5.09 - ETA: 18:54 - loss: 4.90 - ETA: 17:29 - loss: 4.72 - ETA: 16:19 - loss: 4.53 - ETA: 15:19 - loss: 4.37 - ETA: 14:28 - loss: 4.22 - ETA: 13:43 - loss: 4.08 - ETA: 13:04 - loss: 3.97 - ETA: 12:29 - loss: 3.86 - ETA: 11:58 - loss: 3.77 - ETA: 11:30 - loss: 3.69 - ETA: 11:04 - loss: 3.61 - ETA: 10:41 - loss: 3.54 - ETA: 10:20 - loss: 3.48 - ETA: 10:00 - loss: 3.42 - ETA: 9:42 - loss: 3.3669 - ETA: 9:26 - loss: 3.328 - ETA: 9:10 - loss: 3.281 - ETA: 8:56 - loss: 3.240 - ETA: 8:42 - loss: 3.202 - ETA: 8:29 - loss: 3.165 - ETA: 8:17 - loss: 3.133 - ETA: 8:06 - loss: 3.104 - ETA: 7:

187/187 [==============================] - ETA: 4:48 - loss: 1.444 - ETA: 4:47 - loss: 1.419 - ETA: 4:46 - loss: 1.460 - ETA: 4:44 - loss: 1.482 - ETA: 4:43 - loss: 1.474 - ETA: 4:41 - loss: 1.526 - ETA: 4:39 - loss: 1.503 - ETA: 4:38 - loss: 1.513 - ETA: 4:37 - loss: 1.496 - ETA: 4:35 - loss: 1.492 - ETA: 4:33 - loss: 1.493 - ETA: 4:32 - loss: 1.485 - ETA: 4:30 - loss: 1.479 - ETA: 4:29 - loss: 1.491 - ETA: 4:27 - loss: 1.486 - ETA: 4:26 - loss: 1.492 - ETA: 4:24 - loss: 1.492 - ETA: 4:23 - loss: 1.483 - ETA: 4:21 - loss: 1.483 - ETA: 4:19 - loss: 1.487 - ETA: 4:18 - loss: 1.487 - ETA: 4:16 - loss: 1.478 - ETA: 4:15 - loss: 1.481 - ETA: 4:13 - loss: 1.478 - ETA: 4:12 - loss: 1.473 - ETA: 4:10 - loss: 1.474 - ETA: 4:09 - loss: 1.479 - ETA: 4:07 - loss: 1.480 - ETA: 4:06 - loss: 1.478 - ETA: 4:04 - loss: 1.485 - ETA: 4:02 - loss: 1.484 - ETA: 4:01 - loss: 1.482 - ETA: 3:59 - loss: 1.477 - ETA: 3:58 - loss: 1.476 - ETA: 3:56 - loss: 1.477 - ETA: 3:55 - loss: 1.478 - ETA: 3:53 - loss: 1.4

187/187 [==============================] - ETA: 4:48 - loss: 1.493 - ETA: 4:46 - loss: 1.503 - ETA: 4:45 - loss: 1.524 - ETA: 4:44 - loss: 1.518 - ETA: 4:43 - loss: 1.501 - ETA: 4:41 - loss: 1.480 - ETA: 4:40 - loss: 1.474 - ETA: 4:38 - loss: 1.449 - ETA: 4:37 - loss: 1.444 - ETA: 4:35 - loss: 1.440 - ETA: 4:33 - loss: 1.459 - ETA: 4:32 - loss: 1.458 - ETA: 4:30 - loss: 1.462 - ETA: 4:29 - loss: 1.460 - ETA: 4:27 - loss: 1.443 - ETA: 4:26 - loss: 1.436 - ETA: 4:24 - loss: 1.432 - ETA: 4:37 - loss: 1.435 - ETA: 4:47 - loss: 1.432 - ETA: 4:58 - loss: 1.422 - ETA: 5:07 - loss: 1.407 - ETA: 5:19 - loss: 1.412 - ETA: 5:28 - loss: 1.411 - ETA: 5:35 - loss: 1.407 - ETA: 5:40 - loss: 1.404 - ETA: 5:45 - loss: 1.406 - ETA: 5:48 - loss: 1.410 - ETA: 5:51 - loss: 1.410 - ETA: 5:57 - loss: 1.410 - ETA: 6:02 - loss: 1.411 - ETA: 6:05 - loss: 1.411 - ETA: 6:05 - loss: 1.409 - ETA: 6:04 - loss: 1.408 - ETA: 6:04 - loss: 1.408 - ETA: 6:03 - loss: 1.409 - ETA: 6:02 - loss: 1.404 - ETA: 5:58 - loss: 1.3

187/187 [==============================] - ETA: 4:48 - loss: 1.499 - ETA: 4:47 - loss: 1.537 - ETA: 4:45 - loss: 1.474 - ETA: 4:43 - loss: 1.440 - ETA: 4:42 - loss: 1.414 - ETA: 4:40 - loss: 1.383 - ETA: 4:39 - loss: 1.347 - ETA: 4:37 - loss: 1.338 - ETA: 4:36 - loss: 1.348 - ETA: 4:34 - loss: 1.360 - ETA: 4:33 - loss: 1.340 - ETA: 4:31 - loss: 1.350 - ETA: 4:29 - loss: 1.337 - ETA: 4:28 - loss: 1.342 - ETA: 4:26 - loss: 1.339 - ETA: 4:25 - loss: 1.336 - ETA: 4:23 - loss: 1.336 - ETA: 4:22 - loss: 1.340 - ETA: 4:20 - loss: 1.342 - ETA: 4:19 - loss: 1.340 - ETA: 4:24 - loss: 1.333 - ETA: 4:35 - loss: 1.341 - ETA: 4:44 - loss: 1.342 - ETA: 4:53 - loss: 1.346 - ETA: 5:00 - loss: 1.342 - ETA: 5:05 - loss: 1.354 - ETA: 5:10 - loss: 1.355 - ETA: 5:14 - loss: 1.353 - ETA: 5:18 - loss: 1.345 - ETA: 5:21 - loss: 1.344 - ETA: 5:24 - loss: 1.345 - ETA: 5:26 - loss: 1.349 - ETA: 5:28 - loss: 1.346 - ETA: 5:30 - loss: 1.345 - ETA: 5:31 - loss: 1.346 - ETA: 5:28 - loss: 1.347 - ETA: 5:26 - loss: 1.3

187/187 [==============================] - ETA: 4:47 - loss: 1.283 - ETA: 4:46 - loss: 1.321 - ETA: 4:45 - loss: 1.249 - ETA: 4:44 - loss: 1.307 - ETA: 4:42 - loss: 1.318 - ETA: 4:40 - loss: 1.307 - ETA: 4:39 - loss: 1.302 - ETA: 4:37 - loss: 1.278 - ETA: 4:36 - loss: 1.296 - ETA: 4:34 - loss: 1.297 - ETA: 4:33 - loss: 1.301 - ETA: 4:31 - loss: 1.302 - ETA: 4:30 - loss: 1.296 - ETA: 4:28 - loss: 1.289 - ETA: 4:26 - loss: 1.294 - ETA: 4:25 - loss: 1.284 - ETA: 4:23 - loss: 1.288 - ETA: 4:22 - loss: 1.283 - ETA: 4:20 - loss: 1.289 - ETA: 4:19 - loss: 1.299 - ETA: 4:17 - loss: 1.292 - ETA: 4:16 - loss: 1.287 - ETA: 4:14 - loss: 1.287 - ETA: 4:12 - loss: 1.289 - ETA: 4:11 - loss: 1.289 - ETA: 4:09 - loss: 1.284 - ETA: 4:31 - loss: 1.280 - ETA: 8:13 - loss: 1.284 - ETA: 8:08 - loss: 1.275 - ETA: 8:03 - loss: 1.276 - ETA: 7:58 - loss: 1.274 - ETA: 7:54 - loss: 1.277 - ETA: 7:49 - loss: 1.281 - ETA: 7:42 - loss: 1.278 - ETA: 7:34 - loss: 1.279 - ETA: 7:27 - loss: 1.276 - ETA: 7:20 - loss: 1.2

...

187/187 [==============================] - ETA: 4:47 - loss: 1.207 - ETA: 4:46 - loss: 1.087 - ETA: 4:45 - loss: 1.020 - ETA: 4:44 - loss: 1.047 - ETA: 4:42 - loss: 1.038 - ETA: 4:40 - loss: 1.032 - ETA: 4:39 - loss: 1.033 - ETA: 4:37 - loss: 1.019 - ETA: 4:36 - loss: 1.017 - ETA: 4:34 - loss: 1.024 - ETA: 4:33 - loss: 1.015 - ETA: 4:31 - loss: 1.009 - ETA: 4:29 - loss: 1.005 - ETA: 4:28 - loss: 1.012 - ETA: 4:26 - loss: 1.015 - ETA: 4:25 - loss: 1.013 - ETA: 4:23 - loss: 1.014 - ETA: 4:22 - loss: 1.009 - ETA: 4:20 - loss: 1.015 - ETA: 4:19 - loss: 1.011 - ETA: 4:17 - loss: 1.019 - ETA: 4:15 - loss: 1.014 - ETA: 4:14 - loss: 1.016 - ETA: 4:12 - loss: 1.016 - ETA: 4:11 - loss: 1.019 - ETA: 4:09 - loss: 1.017 - ETA: 4:08 - loss: 1.014 - ETA: 4:06 - loss: 1.005 - ETA: 4:05 - loss: 1.005 - ETA: 4:03 - loss: 1.002 - ETA: 4:01 - loss: 1.002 - ETA: 4:00 - loss: 0.998 - ETA: 3:58 - loss: 0.997 - ETA: 3:57 - loss: 0.994 - ETA: 3:55 - loss: 0.997 - ETA: 3:54 - loss: 0.997 - ETA: 3:52 - loss: 0.9

187/187 [==============================] - ETA: 4:47 - loss: 1.013 - ETA: 4:46 - loss: 1.004 - ETA: 4:44 - loss: 0.990 - ETA: 4:43 - loss: 1.021 - ETA: 4:41 - loss: 1.026 - ETA: 4:40 - loss: 1.026 - ETA: 4:38 - loss: 1.045 - ETA: 4:37 - loss: 1.031 - ETA: 4:35 - loss: 1.027 - ETA: 4:34 - loss: 1.021 - ETA: 4:32 - loss: 1.018 - ETA: 4:31 - loss: 1.020 - ETA: 4:29 - loss: 1.019 - ETA: 4:27 - loss: 1.010 - ETA: 4:26 - loss: 1.005 - ETA: 4:24 - loss: 1.002 - ETA: 4:23 - loss: 0.993 - ETA: 4:21 - loss: 0.996 - ETA: 4:20 - loss: 0.988 - ETA: 4:18 - loss: 0.986 - ETA: 4:17 - loss: 0.984 - ETA: 4:15 - loss: 0.979 - ETA: 4:14 - loss: 0.986 - ETA: 4:12 - loss: 0.985 - ETA: 4:10 - loss: 0.983 - ETA: 4:09 - loss: 0.977 - ETA: 4:07 - loss: 0.978 - ETA: 4:06 - loss: 0.978 - ETA: 4:04 - loss: 0.976 - ETA: 4:03 - loss: 0.975 - ETA: 4:01 - loss: 0.981 - ETA: 4:00 - loss: 0.979 - ETA: 3:58 - loss: 0.977 - ETA: 3:57 - loss: 0.976 - ETA: 3:55 - loss: 0.976 - ETA: 3:53 - loss: 0.971 - ETA: 3:52 - loss: 0.9

187/187 [==============================] - ETA: 5:01 - loss: 0.927 - ETA: 4:59 - loss: 1.013 - ETA: 4:54 - loss: 1.015 - ETA: 4:50 - loss: 1.036 - ETA: 4:47 - loss: 1.030 - ETA: 4:45 - loss: 1.022 - ETA: 4:43 - loss: 0.995 - ETA: 4:41 - loss: 0.988 - ETA: 4:39 - loss: 0.982 - ETA: 4:37 - loss: 0.978 - ETA: 4:35 - loss: 0.991 - ETA: 4:33 - loss: 0.998 - ETA: 4:31 - loss: 0.993 - ETA: 4:30 - loss: 0.993 - ETA: 4:28 - loss: 1.000 - ETA: 4:26 - loss: 1.006 - ETA: 4:25 - loss: 1.002 - ETA: 4:23 - loss: 1.002 - ETA: 4:21 - loss: 0.997 - ETA: 4:20 - loss: 0.996 - ETA: 4:18 - loss: 0.996 - ETA: 4:16 - loss: 1.002 - ETA: 4:15 - loss: 0.995 - ETA: 4:13 - loss: 1.001 - ETA: 4:12 - loss: 0.999 - ETA: 4:10 - loss: 0.997 - ETA: 4:09 - loss: 1.000 - ETA: 4:07 - loss: 0.994 - ETA: 4:05 - loss: 0.991 - ETA: 4:04 - loss: 0.993 - ETA: 4:02 - loss: 0.989 - ETA: 4:01 - loss: 0.988 - ETA: 3:59 - loss: 0.986 - ETA: 3:57 - loss: 0.983 - ETA: 3:56 - loss: 0.987 - ETA: 3:54 - loss: 0.985 - ETA: 3:53 - loss: 0.9

187/187 [==============================] - ETA: 4:49 - loss: 1.192 - ETA: 4:46 - loss: 1.060 - ETA: 4:45 - loss: 1.051 - ETA: 4:43 - loss: 1.072 - ETA: 4:42 - loss: 1.042 - ETA: 4:40 - loss: 1.009 - ETA: 4:39 - loss: 1.011 - ETA: 4:37 - loss: 0.997 - ETA: 4:36 - loss: 0.992 - ETA: 4:34 - loss: 0.999 - ETA: 4:32 - loss: 0.990 - ETA: 4:41 - loss: 0.987 - ETA: 15:37 - loss: 0.98 - ETA: 17:03 - loss: 0.98 - ETA: 16:47 - loss: 0.97 - ETA: 16:32 - loss: 0.97 - ETA: 15:51 - loss: 0.97 - ETA: 15:12 - loss: 0.97 - ETA: 14:38 - loss: 0.97 - ETA: 14:06 - loss: 0.97 - ETA: 13:38 - loss: 0.96 - ETA: 13:12 - loss: 0.97 - ETA: 12:48 - loss: 0.96 - ETA: 12:24 - loss: 0.96 - ETA: 12:01 - loss: 0.96 - ETA: 11:40 - loss: 0.98 - ETA: 11:20 - loss: 0.98 - ETA: 11:01 - loss: 0.98 - ETA: 10:44 - loss: 0.98 - ETA: 10:28 - loss: 0.98 - ETA: 10:12 - loss: 0.98 - ETA: 9:58 - loss: 0.9828 - ETA: 9:44 - loss: 0.984 - ETA: 9:31 - loss: 0.980 - ETA: 9:18 - loss: 0.978 - ETA: 9:06 - loss: 0.973 - ETA: 8:55 - loss: 0.

187/187 [==============================] - ETA: 4:47 - loss: 0.945 - ETA: 4:47 - loss: 0.937 - ETA: 4:45 - loss: 0.931 - ETA: 4:43 - loss: 0.967 - ETA: 4:42 - loss: 0.995 - ETA: 4:40 - loss: 1.017 - ETA: 4:39 - loss: 1.031 - ETA: 4:37 - loss: 1.043 - ETA: 4:35 - loss: 1.035 - ETA: 4:34 - loss: 1.024 - ETA: 4:32 - loss: 1.033 - ETA: 4:31 - loss: 1.014 - ETA: 4:29 - loss: 1.009 - ETA: 4:27 - loss: 1.011 - ETA: 4:26 - loss: 1.008 - ETA: 4:24 - loss: 1.012 - ETA: 4:23 - loss: 1.009 - ETA: 4:21 - loss: 0.996 - ETA: 4:20 - loss: 0.995 - ETA: 4:18 - loss: 0.993 - ETA: 4:17 - loss: 0.990 - ETA: 4:15 - loss: 0.991 - ETA: 4:14 - loss: 0.989 - ETA: 4:12 - loss: 0.989 - ETA: 4:10 - loss: 0.989 - ETA: 4:09 - loss: 0.991 - ETA: 4:07 - loss: 0.985 - ETA: 4:06 - loss: 0.979 - ETA: 4:04 - loss: 0.981 - ETA: 4:03 - loss: 0.981 - ETA: 4:01 - loss: 0.979 - ETA: 4:00 - loss: 0.979 - ETA: 3:58 - loss: 0.971 - ETA: 3:57 - loss: 0.969 - ETA: 3:55 - loss: 0.973 - ETA: 3:53 - loss: 0.973 - ETA: 3:52 - loss: 0.9

In [23]:
print("Time for training: {} seconds".format(time_train))

Time for training: 6747.911556720734 seconds


In [24]:
if not os.path.exists(config.PATH.CALLBACKS_PATH):
    os.mkdir(config.PATH.CALLBACKS_PATH)   
callback_df = pd.DataFrame(callbacks.history)
callback_df.to_csv(callbacks_path, index=None)